### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до **2го знака**.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1 минута
Одновременно минимум, максимум и медиана:easy - 3 минуты
medium - 6 минут
nightmare - 6 минут


In [2]:
import numpy as np

#### Генерация данных

In [3]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

In [4]:
N, W = 10000000, 1000
def data_stream(n):
    random_generator = random.Random(42)
    easy = 0
    #for _ in range(10000000):
    for _ in range(n):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )

In [5]:
def easy_stream(n):
    for record in data_stream(n):
        yield record.easy
        
def medium_stream(n):
    for record in data_stream(n):
        yield record.medium
        
def nightmare_stream(n):
    for record in data_stream(n):
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [6]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(x) for x in result / count]

In [7]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print(get_tuple_stream_mean(example(medium_stream(100*W)), 2))

['127.75', '137.75']
CPU times: user 2.59 s, sys: 0 ns, total: 2.59 s
Wall time: 2.96 s


In [8]:
from collections import deque
DEBUG = True

In [9]:
list(medium_stream(5))

[57, 140, 71, 44, 16]

In [10]:
def mean_var(w, stream):
    q = deque()
    valsum, sqsum = 0, 0
    for val in stream:
        valsum += val
        sqsum += val*val
        q.append(val)
        if len(q) < w: continue
        if len(q) > w:
            oldval = q.popleft()
            valsum -= oldval
            sqsum -= oldval*oldval
        #if DEBUG:
        #    arr = np.array(q)
        #    assert( np.isclose( arr.mean(), valsum/w ) )
        #    assert( np.isclose( arr.var(), sqsum/w - valsum**2/w**2 ) )
        yield (valsum/w, sqsum/w - valsum**2/w**2)
            

In [11]:
%%time
DEBUG = False
print(get_tuple_stream_mean(mean_var(W, medium_stream(100*W)), 2))

['127.74', '5475.85']
CPU times: user 2.84 s, sys: 15.6 ms, total: 2.86 s
Wall time: 3.34 s


In [11]:
%%time
DEBUG = False
print(get_tuple_stream_mean(mean_var(W, easy_stream(N)), 2))

['4999675.28', '83439.34']
CPU times: user 5min 4s, sys: 2.7 s, total: 5min 6s
Wall time: 5min 31s


In [12]:
%%time
DEBUG = False
print(get_tuple_stream_mean(mean_var(W, medium_stream(N)), 2))

['127.48', '5455.17']
CPU times: user 4min 52s, sys: 2.24 s, total: 4min 54s
Wall time: 5min 12s


In [13]:
%%time
DEBUG = False
print(get_tuple_stream_mean(mean_var(W, nightmare_stream(N)), 2))

['499880345.88', '83228908564027760.00']
CPU times: user 5min 7s, sys: 2.82 s, total: 5min 10s
Wall time: 5min 30s


In [12]:
def min_max_median_easy(w, stream):
    q = deque()
    for val in stream:
        q.append(val)
        if len(q) < w: continue
        if len(q) > w:
            _ = q.popleft()
        yield (q[0], q[-1], q[len(q) // 2])
            

In [15]:
%%time
DEBUG = False
print(get_tuple_stream_mean(min_max_median_easy(W, easy_stream(N)), 3))

['4999175.79', '5000174.76', '4999675.78']
CPU times: user 4min 41s, sys: 3.56 s, total: 4min 44s
Wall time: 5min 27s


In [13]:
from sortedcontainers import SortedList

In [14]:
def min_max_median(w, stream):
    q = deque()
    sl = SortedList()
    for val in stream:
        q.append(val)
        sl.add(val)
        if len(q) < w: continue
        if len(q) > w:
            oldval = q.popleft()
            sl.remove(oldval)
        yield (sl[0], sl[-1], sl[len(sl) // 2])
            

In [17]:
%%time
DEBUG = False
print(get_tuple_stream_mean(min_max_median(W, easy_stream(N)), 3))

['4999175.79', '5000174.76', '4999675.78']
CPU times: user 6min 54s, sys: 1.98 s, total: 6min 56s
Wall time: 7min 13s


In [15]:
%%time
DEBUG = False
print(get_tuple_stream_mean(min_max_median(W, medium_stream(N)), 3))

['0.02', '254.98', '127.60']
CPU times: user 6min 58s, sys: 2.32 s, total: 7min
Wall time: 7min 23s


In [16]:
%%time
DEBUG = False
print(get_tuple_stream_mean(min_max_median(W, nightmare_stream(N)), 3))

['1017512.29', '999017359.97', '500438415.64']
CPU times: user 7min, sys: 2.1 s, total: 7min 3s
Wall time: 7min 25s
